In [38]:
#hal pertama yang dilakukan
import pandas as pd
import numpy as np
import scipy as sp

#plotting
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#decisiontree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

#imputersist untuk mengisi missing value
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer

#hash hampir lupa
#Support Vector Machines
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_csv('train.csv')

In [3]:
# melihat semua yang ada dalam dataset
print(data.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#mari kita cek sama2

data.info()
print("_" * 40)

#tipedata yang nanti digunakan harus bersifat float. sebelumnya ada error disini

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
________________________________________


In [6]:
#mengkorelasikan beberapa kolom yang insyaAllah bakal digunakan '-'
#di sesuaikan per nilai isi atribut tsb (YANG PALING TINGGI NILAI HIDUPNYA)
#pcclass duls sist

data[['Pclass', 'Survived']].groupby(['Pclass'] , as_index=False).mean().sort_values(by='Survived',ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [7]:
#lanjut ke jenis kelamin

data[['Sex', 'Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [8]:
#SibSp
#perlu diingat ini di urutkan dr survive "FALSE" ato yg paling tinggi tingkat hidupnye

data[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)


,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [9]:
#parch sist

data[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [10]:
#disini saya memutuskan untuk menghilangkan kolom cabin & tiket karna ada banyak data yg hilang serta 
#nantinya dapat berpengaruh dengan hasil akurasi yang kita dapatkan

data = data.drop(['Ticket', 'Cabin'], axis=1)

In [11]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [12]:
#split title seperti modul sebelume

data.insert(value=data.Name.map(lambda name: name.split(",")[1].split(".")[0].strip()),loc=10,column="Title")

In [13]:
title_map={"Capt":"Officer",
          "Col":"Officer",
          "Major":"Officer",
          "Johkheer":"Royalty",
          "Don":"Royalty",
          "Sir":"Royalty",
          "Dr":"Royalty",
          "Rev":"Officer",
          "The Countess":"Royalty",
          "Dona":"Royalty",
          "Mme":"Mrs",
          "Mlle":"Miss",
          "Ms":"Mrs",
           "Mr":"Mr",
           "Mrs":"Mrs",
           "Miss":"Miss",
           "Master":"Master",
           "Lady":"Royalty" }

data["Title"]=data.Title.map(title_map)

In [14]:
#title sist

data[['Title', 'Survived']].groupby(['Title'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Title,Survived
3,Mrs,0.795276
1,Miss,0.701087
0,Master,0.575000
5,Royalty,0.500000
4,Officer,0.181818
2,Mr,0.156673


In [15]:
#selanjutnya kita akan mengubah data title yang categorical tadi menjadi data ordinal untuk memudahkan perhitungan

data['Title'] = pd.factorize(data.Title)[0]
data['Title'].unique()

array([ 0,  1,  2,  3,  4,  5, -1], dtype=int64)

In [16]:
#title sist

data[['Title', 'Survived']].groupby(['Title'], as_index=False).mean().sort_values(by='Survived', ascending=False)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,0


In [17]:
#title beres. Name&PassengerID di drop ya!!!

data = data.drop(['Name', 'PassengerId'], axis=1)

In [18]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,0
1,1,1,female,38.0,1,0,71.2833,C,1
2,1,3,female,26.0,0,0,7.9250,S,2
3,1,1,female,35.0,1,0,53.1000,S,1
4,0,3,male,35.0,0,0,8.0500,S,0


In [19]:
#merubah nilai jenis kelamin juga

data['Sex'] = pd.factorize(data.Title)[0]
data['Sex'].unique()
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,S,0
1,1,1,1,38.0,1,0,71.2833,C,1
2,1,3,2,26.0,0,0,7.9250,S,2
3,1,1,1,35.0,1,0,53.1000,S,1
4,0,3,0,35.0,0,0,8.0500,S,0


In [20]:
#cek missing values

data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
Title         0
dtype: int64

In [21]:
#Karena ada beberapa data di kolom age yang kosong dan kita memerlukan kolom age untuk model kita karena memiliki 
#korelasi yang besar, maka kita perlu untuk mengisi age yang kosong tadi
#menggunakan library simpleimputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(data[['Age']])
data[['Age']] = imputer.transform(data[['Age']])

In [22]:
#cek missing values

data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
Title       0
dtype: int64

In [23]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,S,0
1,1,1,1,38.0,1,0,71.2833,C,1
2,1,3,2,26.0,0,0,7.9250,S,2
3,1,1,1,35.0,1,0,53.1000,S,1
4,0,3,0,35.0,0,0,8.0500,S,0


In [24]:
#merubah nilai AGE juga

data['Age'] = pd.factorize(data.Title)[0]
data['Age'].unique()
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,0,1,0,7.2500,S,0
1,1,1,1,1,1,0,71.2833,C,1
2,1,3,2,2,0,0,7.9250,S,2
3,1,1,1,1,1,0,53.1000,S,1
4,0,3,0,0,0,0,8.0500,S,0


In [25]:
#membuat family size

data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived',ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [26]:
#buat isAlone yang sendirian diambil dr familysize

data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1
    
data[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [27]:
#drop kolom-kolom yang tidak kita pakai

data = data.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,0,7.2500,S,0,0
1,1,1,1,1,71.2833,C,1,0
2,1,3,2,2,7.9250,S,2,1
3,1,1,1,1,53.1000,S,1,0
4,0,3,0,0,8.0500,S,0,1


In [28]:
#merubah nilai EMBARKED juga

data['Embarked'] = pd.factorize(data.Title)[0]
data['Embarked'].unique()
data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,0,7.2500,0,0,0
1,1,1,1,1,71.2833,1,1,0
2,1,3,2,2,7.9250,2,2,1
3,1,1,1,1,53.1000,1,1,0
4,0,3,0,0,8.0500,0,0,1


In [29]:
#mengisi missing value EMBARKED

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(data[['Embarked']])
data[['Embarked']] = imputer.transform(data[['Embarked']])

In [30]:
#cek missing values

data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
Title       0
IsAlone     0
dtype: int64

In [31]:
#hampir lupa kolom fare belum diubah jadi numeric

data['Fare'] = pd.factorize(data.Title)[0]
data['Fare'].unique()
data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,0,0,0.0,0,0
1,1,1,1,1,1,1.0,1,0
2,1,3,2,2,2,2.0,2,1
3,1,1,1,1,1,1.0,1,0
4,0,3,0,0,0,0.0,0,1


In [32]:
#memisahkan kolom

xdata = data.drop('Survived', axis=1)
ydata = data.Survived

In [33]:
#memisahkan data 

xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size=0.3, random_state=1)

In [34]:
# implementasi sist Decision Tree

clf=DecisionTreeClassifier()
clf=clf.fit(xtrain,ytrain)


In [35]:
#memprediksi data uji

ypred = clf.predict(xtest)

In [80]:
#akurasi

akurasidecision = (metrics.accuracy_score(ytest,ypred)*100)
print("Akurasi Decision Tree: ", akurasidecision) 

Akurasi Decision Tree:  74.25373134328358


In [81]:
# #implementasi Random Forest (selain decision sist)

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(xtrain, ytrain)
ypred = random_forest.predict(xtest)
random_forest.score(xtrain, ytrain)
akurasirandom = (random_forest.score(xtrain, ytrain) * 100)
print("Akurasi Random Forest: " , akurasirandom )

Akurasi Random Forest:  82.98555377207063


In [78]:
#implementasi SVM (Support Vector Machines)

svc = SVC()
svc.fit(xtrain, ytrain)
ypred = svc.predict(xtest)
akurasiSVM = (svc.score(xtrain, ytrain)*100)
print("Akurasi SVM: " , akurasiSVM)

#akurasiSVM = print("Akurasi SVM: " ,svc.score(xtrain, ytrain)*100)
#akurasiSVM

Akurasi SVM:  81.38041733547352


In [82]:
#yuk mari kita bandingin sist

models = pd.DataFrame({
    'Model': ['Decision Tree', 'Random Forest','Support Vector Machine'],
    'Score': [akurasidecision, akurasirandom, akurasiSVM]
})
models.sort_values(by="Score", ascending=False)

,Model,Score
1,Random Forest,82.985554
2,Support Vector Machine,81.380417
0,Decision Tree,74.253731


# Algoritma yang memiliki tingkat akurasi paling tinggi ialah Random Forest. Sekian^^